# Aufbau von KI-Agenten mit persistentem Gedächtnis mithilfe von Cognee

Dieses Notebook zeigt, wie man intelligente KI-Agenten mit fortschrittlichen Gedächtnisfähigkeiten mithilfe von [**cognee**](https://www.cognee.ai/) erstellt – einem Open-Source-KI-Gedächtnis, das Wissensgraphen, semantische Suche und Sitzungsmanagement kombiniert, um kontextbewusste KI-Systeme zu entwickeln.

## 🎯 Lernziele

Am Ende dieses Tutorials wirst du verstehen, wie man:
- **Wissensgraphen basierend auf Embeddings erstellt**: Unstrukturierte Texte in strukturierte, abfragbare Wissensdaten umwandelt
- **Sitzungsgedächtnis implementiert**: Mehrstufige Gespräche mit automatischer Kontextbeibehaltung erstellt
- **Gespräche speichert**: Wichtige Interaktionen optional im Langzeitgedächtnis für zukünftige Verwendungen aufbewahrt
- **Mit natürlicher Sprache abfragt**: Historischen Kontext in neuen Gesprächen nutzt und darauf zugreift
- **Gedächtnis visualisiert**: Die Beziehungen im Wissensgraphen deines Agenten erkundet


## 🏗️ Was Sie bauen werden

In diesem Tutorial erstellen wir einen **Coding-Assistenten** mit persistentem Speicher, der:

### 1. **Wissensdatenbank-Erstellung**
   - Entwicklerprofile und Fachkenntnisse aufnimmt
   - Prinzipien und Best Practices der Python-Programmierung verarbeitet
   - Historische Gespräche zwischen Entwicklern und KI-Assistenten speichert

### 2. **Sitzungsbewusste Gespräche**
   - Kontext über mehrere Fragen in derselben Sitzung beibehält
   - Jedes Frage-/Antwort-Paar automatisch zwischenspeichert, um eine effiziente Abrufbarkeit zu gewährleisten
   - Kohärente, kontextbezogene Antworten basierend auf der Gesprächshistorie liefert

### 3. **Langzeitgedächtnis**
   - Wichtige Gespräche in ein Langzeitgedächtnis speichert
   - Relevante Erinnerungen aus der Wissensdatenbank und früheren Sitzungen abruft, um neue Interaktionen zu informieren
   - Eine wachsende Wissensdatenbank aufbaut, die sich im Laufe der Zeit verbessert

### 4. **Intelligente Speicherabrufung**
   - Graph-bewusste semantische Suche verwendet, um relevante Informationen in allen gespeicherten Daten zu finden
   - Suchanfragen nach Datenuntergruppen filtert (Entwicklerinformationen vs. Prinzipien)
   - Mehrere Datenquellen kombiniert, um umfassende Antworten zu liefern


## 📋 Voraussetzungen & Einrichtung

### Systemanforderungen

Bevor Sie beginnen, stellen Sie sicher, dass Sie Folgendes haben:

1. **Python-Umgebung**
   - Python 3.9 oder höher
   - Virtuelle Umgebung (empfohlen)
   
2. **Redis-Cache** (Erforderlich für Sitzungsverwaltung)
   - Lokales Redis: `docker run -d -p 6379:6379 redis`
   - Oder nutzen Sie einen verwalteten Redis-Dienst
   
3. **LLM-API-Zugriff**
   - OpenAI-API-Schlüssel oder andere Anbieter (siehe [Dokumentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Datenbankkonfiguration**
   - Standardmäßig ist keine Konfiguration erforderlich. Cognee verwendet dateibasierte Datenbanken (LanceDB und Kuzu)
   - Optional können Sie Azure AI Search als Vektorspeicher einrichten (siehe [Dokumentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Umgebungs-Konfiguration

Erstellen Sie eine `.env`-Datei in Ihrem Projektverzeichnis mit den folgenden Variablen:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Verständnis der Speicherarchitektur von Cognee

### Wie Cognee funktioniert

Cognee bietet ein ausgeklügeltes Speichersystem, das über einfache Schlüssel-Wert-Speicherung hinausgeht:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Hauptkomponenten:

1. **Wissensgraph**: Speichert Entitäten, Beziehungen und semantische Verbindungen
2. **Vektoreinbettungen**: Ermöglicht semantische Suche über alle gespeicherten Informationen
3. **Sitzungscache**: Hält den Gesprächskontext innerhalb und zwischen Sitzungen aufrecht
4. **NodeSets**: Organisieren Daten in logische Kategorien für gezielte Abrufe

### Speicherarten in diesem Tutorial:

- **Persistenter Speicher**: Langzeitspeicherung im Wissensgraph
- **Sitzungsspeicher**: Temporärer Gesprächskontext im Redis-Cache
- **Semantischer Speicher**: Vektorbasierte Ähnlichkeitssuche über alle Daten hinweg


## 📦 Erforderliche Pakete installieren

Installieren Sie Cognee mit Redis-Unterstützung für die Sitzungsverwaltung:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Umgebung initialisieren und Bibliotheken laden

Stellen Sie sicher:
1. Redis läuft (z. B. über Docker: `docker run -d -p 6379:6379 redis`)
2. Umgebungsvariablen sind gesetzt, bevor Cache-Module importiert werden
3. Falls nötig, starten Sie den Kernel neu und führen Sie die Zellen der Reihe nach aus

Die folgende Zelle wird:
1. Umgebungsvariablen aus `.env` laden
2. Cognee mit Ihren LLM-Einstellungen konfigurieren
3. Caching für das Sitzungsmanagement aktivieren
4. Überprüfen, ob alle Komponenten ordnungsgemäß verbunden sind


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Speicherverzeichnisse konfigurieren

Cognee verwendet zwei separate Verzeichnisse für seine Operationen:
- **Daten-Root**: Speichert eingelesene Dokumente und verarbeitete Daten
- **System-Root**: Enthält die Wissensgraph-Datenbank und System-Metadaten

Für dieses Tutorial erstellen wir isolierte Verzeichnisse wie folgt:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Speicherzustand zurücksetzen

Bevor wir mit dem Aufbau unseres Speichersystems beginnen, stellen wir sicher, dass wir mit einem sauberen Start beginnen.

> 💡 **Tipp**: Sie können diesen Schritt überspringen, wenn Sie bestehende Erinnerungen aus Ihren vorherigen Durchläufen beibehalten möchten, wenn Sie dieses Notebook später verwenden.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Teil 1: Aufbau der Wissensdatenbank

### Datenquellen für unseren Entwicklerassistenten

Wir werden drei Arten von Daten einbinden, um eine umfassende Wissensdatenbank zu erstellen:

1. **Entwicklerprofil**: Persönliche Expertise und technischer Hintergrund  
2. **Python Best Practices**: Die Zen-Prinzipien von Python mit praktischen Richtlinien  
3. **Historische Gespräche**: Frühere Q&A-Sitzungen zwischen Entwicklern und KI-Assistenten  

Diese vielfältigen Daten ermöglichen es unserem Agenten:  
- Den technischen Kontext des Nutzers zu verstehen  
- Best Practices in Empfehlungen anzuwenden  
- Aus früheren erfolgreichen Interaktionen zu lernen  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Daten in Wissensgraph umwandeln

Jetzt werden wir unseren Rohtext in eine strukturierte Erinnerung umwandeln. Dieser Prozess:

1. **Fügt Daten zu NodeSets hinzu**: Organisiert Informationen in logische Kategorien
   - `developer_data`: Entwicklerprofile und Gespräche
   - `principles_data`: Python-Best Practices und Richtlinien

2. **Führt die Cognify-Pipeline aus**: Extrahiert Entitäten, Beziehungen und erstellt Embeddings
   - Identifiziert Schlüsselkonzepte
   - Erstellt semantische Verbindungen zwischen verwandten Informationen
   - Generiert Vektor-Embeddings

Dies kann einige Momente dauern, während das LLM den Text verarbeitet und die Graphstruktur erstellt:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualisieren des Wissensgraphen

Lassen Sie uns die Struktur unseres Wissensgraphen erkunden. Die Visualisierung zeigt:
- **Knoten**: Aus dem Text extrahierte Entitäten (Konzepte, Technologien, Personen)
- **Kanten**: Beziehungen und Verbindungen zwischen den Entitäten
- **Cluster**: Verwandte Konzepte, gruppiert nach semantischer Ähnlichkeit

Öffnen Sie die generierte HTML-Datei in Ihrem Browser, um den Graphen interaktiv zu erkunden:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Gedächtnis mit Memify bereichern

Die Funktion `memify()` analysiert den Wissensgraphen und generiert intelligente Regeln über die Daten. Dieser Prozess:
- Identifiziert Muster und bewährte Praktiken
- Erstellt umsetzbare Richtlinien basierend auf dem Inhalt
- Stellt Beziehungen zwischen verschiedenen Wissensbereichen her

Diese Regeln helfen dem Agenten, fundiertere Entscheidungen zu treffen, wenn Fragen beantwortet werden. Das Erfassen einer zweiten Visualisierung ermöglicht es Ihnen, zu vergleichen, wie der Graph nach der Anreicherung dichter wird.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Teil 2: Intelligente Speicherabfrage

### Demonstration 1: Wissensintegration über Dokumente hinweg

Jetzt, da unser Wissensgraph erstellt ist, testen wir, wie Cognee Informationen aus mehreren Quellen kombiniert, um komplexe Fragen zu beantworten.

Die erste Abfrage demonstriert:
- **Semantisches Verständnis**: Relevante Konzepte finden, auch wenn sie nicht explizit erwähnt werden
- **Querverweise**: Verknüpfung von Entwicklerprofilen mit Python-Prinzipien
- **Kontextuelles Denken**: Anwendung von Best Practices auf spezifische Implementierungen

### Demonstration 2: Gefilterte Suche mit NodeSets

Die zweite Abfrage zeigt, wie spezifische Teilmengen des Wissensgraphen gezielt durchsucht werden können:
- Verwendet den Parameter `node_name`, um nur innerhalb von `principles_data` zu suchen
- Liefert fokussierte Antworten aus einem spezifischen Wissensbereich
- Nützlich, wenn domänenspezifische Informationen benötigt werden


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Teil 3: Einrichtung des Sitzungsmanagements

### Aktivieren der Gesprächsspeicherung

Das Sitzungsmanagement ist entscheidend, um den Kontext über mehrere Interaktionen hinweg beizubehalten. Hier werden wir:

1. **Benutzerkontext initialisieren**: Ein Benutzerprofil erstellen oder abrufen, um die Sitzung zu verfolgen
2. **Cache-Engine konfigurieren**: Verbindung zu Redis herstellen, um den Gesprächsverlauf zu speichern
3. **Sitzungsvariablen aktivieren**: Kontextvariablen einrichten, die über Abfragen hinweg bestehen bleiben

> ⚠️ **Wichtig**: Dafür muss Redis laufen und `CACHING=true` in Ihrer Umgebung gesetzt sein


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hilfsfunktion: Sitzungsverlauf anzeigen

Diese Hilfsfunktion ermöglicht es uns, den in Redis gespeicherten Gesprächsverlauf zu überprüfen. Sie ist nützlich für:
- Debugging der Sitzungsverwaltung
- Überprüfung, ob Gespräche zwischengespeichert werden
- Verständnis, welcher Kontext dem Agenten zur Verfügung steht


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sitzung 1: Async Support Lab — Erste Frage

Starten Sie die `async-support-lab`-Sitzung, indem Sie nach telemetrie-freundlichen asyncio-Mustern für einen massiven Web-Scraper fragen. Der Graph kennt bereits asyncio, aiohttp und Überwachungspraktiken, daher sollte die Antwort frühere Gespräche widerspiegeln und gleichzeitig auf die neue Anfrage zugeschnitten sein.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Speicher von Sitzung 1 nach dem ersten Austausch überprüfen

Das Ausführen von `show_history(session_1)` direkt nach der ersten Frage bestätigt, dass Cognee sowohl die Eingabeaufforderung als auch die Antwort in Redis geschrieben hat. Sie sollten einen Eintrag mit der Parallelitätsanleitung sehen.


In [ ]:
await show_history(session_1)

## Sitzung 1: Nachverfolgung von Datenmodellen

Als Nächstes fragen wir: "Wann sollte ich dataclasses gegenüber Pydantic wählen?" unter Verwendung derselben Sitzungs-ID. Cognee sollte die Python-Prinzipien sowie frühere FastAPI-Gespräche zusammenführen, um differenzierte Ratschläge zu geben – und damit zeigen, dass der Kontext innerhalb einer benannten Sitzung erhalten bleibt.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Bestätigen, dass der Verlauf von Sitzung 1 beide Dialoge enthält

Ein weiterer Aufruf von `show_history(session_1)` sollte zwei Q&A-Einträge anzeigen. Dies entspricht dem "Memory Replay"-Schritt des Mem0-Labors und beweist, dass zusätzliche Dialoge das gleiche Transkript erweitern.


In [ ]:
await show_history(session_1)

## Sitzung 2: Design-Review-Thread — Neue Sitzung

Um die Trennung zwischen Threads zu zeigen, starten wir `design-review-session` und bitten um Protokollierungsrichtlinien für Vorfallbewertungen. Obwohl die zugrunde liegende Wissensbasis dieselbe ist, sorgt die neue Sitzungs-ID dafür, dass Transkripte getrennt bleiben.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Überprüfungssitzung 2 Geschichte

`show_history(session_2)` sollte nur das Design-Review-Prompt/Antwort-Paar auflisten. Vergleichen Sie es mit Sitzung 1, um hervorzuheben, wie Cognee unabhängige Transkripte beibehält, während das gemeinsame Wissensgraphen genutzt wird.


In [ ]:
await show_history(session_2)

## Zusammenfassung

Herzlichen Glückwunsch! Sie haben Ihrem Coding-Assistenten gerade eine echte Langzeitspeicherschicht mit Cognee hinzugefügt.

In diesem Tutorial haben Sie rohe Entwicklerinhalte (Code, Dokumentationen, Chats) in ein Graph- und Vektorspeicherformat umgewandelt, das Ihr Agent durchsuchen, analysieren und kontinuierlich verbessern kann.

Was Sie gelernt haben:

1. **Von rohem Text zu KI-Speicher**: Wie Cognee unstrukturierte Daten aufnimmt und sie mithilfe einer kombinierten Vektor- und Wissensgraph-Architektur in intelligenten, durchsuchbaren Speicher umwandelt.

2. **Graph-Anreicherung mit memify**: Wie Sie über die grundlegende Graph-Erstellung hinausgehen und memify nutzen können, um abgeleitete Fakten und reichhaltigere Beziehungen zu Ihrem bestehenden Graphen hinzuzufügen.

3. **Verschiedene Suchstrategien**: Wie Sie den Speicher mit unterschiedlichen Suchtypen abfragen können (graph-basiertes Q&A, RAG-ähnliche Vervollständigung, Einblicke, rohe Textabschnitte, Codesuche usw.), je nachdem, was Ihr Agent benötigt.

4. **Visuelle Erkundung**: Wie Sie mit Graph-Visualisierungen und der Cognee-Benutzeroberfläche inspizieren und debuggen können, was Cognee erstellt hat, sodass Sie tatsächlich sehen können, wie Wissen strukturiert ist.

5. **Sitzungsbewusster Speicher**: Wie Sie kontextbezogene Sitzungsdaten mit persistentem semantischem Speicher kombinieren können, damit Agenten sich über mehrere Sitzungen hinweg erinnern können, ohne Informationen zwischen Benutzern zu vermischen.


## Wichtige Erkenntnisse
1. Speicher als Wissensgraph unterstützt durch Embeddings

    - **Strukturierte Verständlichkeit**: Cognee kombiniert einen Vektorspeicher und einen Graphspeicher, sodass Ihre Daten sowohl nach Bedeutung durchsuchbar als auch durch Beziehungen verbunden sind. Standardmäßig verwendet Cognee dateibasierte Datenbanken (LanceDB für Vektoren, Kuzu für Graphdatenbanken).

    - **Beziehungsbewusste Suche**: Antworten basieren nicht nur auf „ähnlichem Text“, sondern auch darauf, wie Entitäten miteinander in Beziehung stehen.

    - **Lebendiger Speicher**: Die Speicherebene entwickelt sich weiter, wächst und bleibt als ein verbundener Graph abfragbar.

2. Such- und Denkmodi
    - **Hybride Suche**: Die Suche kombiniert Vektorsimilarität, Graphstruktur und LLM-Logik, von der Suche nach Rohdaten bis hin zu graphbewussten Frage-Antworten.

    - **Den Modus an die Aufgabe anpassen**: Verwenden Sie Modi im Stil von Vervollständigungen, wenn Sie Antworten in natürlicher Sprache möchten, und Chunk-/Zusammenfassungs-/Graph-Modi, wenn Ihr Agent Rohkontext benötigt oder eigene Schlussfolgerungen ziehen soll.

3. Personalisierte, sitzungsbewusste Agenten
    - **Sitzungskontext + Langzeitspeicher**: Cognee trennt den kurzfristigen „Thread“-Kontext vom langfristigen Speicher auf Benutzer- oder Organisationsebene.

## Anwendungen in der Praxis

1. **Vertikale KI-Agenten**

    Nutzen Sie das Muster aus diesem Notebook, um domänenspezifische Copiloten zu entwickeln, die Cognee als Kern für Abruf und Logik verwenden:

- **Entwickler-Copiloten**: Code-Review, Vorfallanalyse und Architekturassistenten, die Code, APIs, Entwurfsdokumente und Tickets als einen einzigen Speichergraphen durchlaufen.

- **Kundenorientierte Copiloten**: Support- oder Erfolgsagenten, die Produktdokumente, FAQs, CRM-Notizen und frühere Tickets mit graphbewusster Suche und zitierten Antworten nutzen.

- **Interne Experten-Copiloten**: Richtlinien-, Rechts- oder Sicherheitsassistenten, die über miteinander verbundene Regeln, Richtlinien und historische Entscheidungen nachdenken, anstatt isolierte PDFs zu verwenden.

    Cognee ist explizit als persistenter, genauer Speicher für KI-Agenten positioniert und bietet einen lebendigen Wissensgraphen, der hinter Ihrem Agenten eingesetzt wird und ad-hoc Kombinationen aus Vektorspeichern und benutzerdefiniertem Graphcode ersetzt.

2. **Daten-Silos in einen Speicher vereinen**

    Der gleiche Ansatz hilft Ihnen, eine einheitliche Speicherebene über verstreute Quellen hinweg aufzubauen:

- **Von Silos zu einem Graphen**: Strukturierte (z. B. Datenbanken) und unstrukturierte Daten (z. B. Dokumente, Chats) in einen einzigen Graphen mit Embeddings einfügen, anstatt separate Indizes für jedes System zu verwenden.

- **Quellenübergreifende Logik mit Zitaten**: Führen Sie mehrstufige Logik über alles aus – „verbinden“ Sie Protokolle, Metriken und Dokumente über den Graphen – und liefern Sie dennoch fundierte Antworten mit Herkunftsnachweisen.

- **Wissenszentren**: Für Bereiche wie Banken oder Bildung wird Cognee bereits genutzt, um PDFs, interne Systeme und App-Daten in einen Wissensgraphen mit Vektoren zu vereinen, sodass Agenten Fragen mit präzisem, zitiertem Kontext beantworten können.

## Nächste Schritte

Sie haben die Kernspeicherschleife implementiert. Hier sind natürliche Erweiterungen, die Sie selbst ausprobieren können (siehe [Cognee-Dokumentation](https://docs.cognee.ai/) für Details):

1. **Mit zeitlicher Bewusstheit experimentieren**: Aktivieren Sie „temporal cognify“, um Ereignisse und Zeitstempel aus Text zu extrahieren.

2. **Ontologie-gesteuerte Logik einführen**: Definieren Sie eine OWL-Ontologie für Ihre Domäne. Nutzen Sie Cognee’s Ontologie-Unterstützung, damit extrahierte Entitäten und Beziehungen in diesem Schema verankert sind, was die Graphqualität und domänenspezifische Antworten verbessert.

3. **Eine Feedback-Schleife hinzufügen**: Lassen Sie Cognee die Gewichtung von Graphkanten basierend auf echtem Benutzerfeedback anpassen, sodass die Suche sich im Laufe der Zeit verbessert, anstatt statisch zu bleiben.

4. **Für Personalisierung & Sitzungsverhalten optimieren**: Verwenden Sie Benutzer-IDs, Mandanten und Datensätze, um jeder Person oder jedem Team eine eigene Ansicht über die gemeinsame Speicher-Engine zu geben.

5. **Auf komplexere Agenten skalieren**: Integrieren Sie Cognee in Agenten-Frameworks, um Multi-Agenten-Systeme zu entwickeln, die alle die gleiche Speicherebene teilen. *Microsoft Agent Framework x Cognee Plugin kommt bald.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
